In [1]:
import os
from transformers import BartTokenizer, BartForConditionalGeneration
import time

In [53]:
tokenizer = BartTokenizer.from_pretrained('distilbart-xsum-12-6')
model = BartForConditionalGeneration.from_pretrained('distilbart-xsum-12-6')

In [1]:
import pandas as pd

path = 'data/'

raw_text = pd.read_csv(path + "news_summary_more.csv", encoding='iso-8859-1')
summary = pd.read_csv(path + "news_summary.csv", encoding='iso-8859-1')

# cnn_text = pd.read_csv(path + "cnn_news.csv", encoding='iso-8859-1')

In [3]:
raw_text.iloc[12321, 1]

"One97 Communications' subsidiary Paytm Payments Bank on Wednesday announced the appointment of Satish Kumar Gupta as the Managing Director (MD) and CEO of the bank. Gupta's appointment comes months after Renu Satti stepped down as the CEO of the Bank to head the company's retail segment. Previously, Gupta worked at National Payments Corporation of India and State Bank of India.  "

In [54]:
def summarize(text, num_beams=4, length_penalty=1.0, max_length=50, min_length=10, no_repeat_ngram_size=3):
  
  text = text.replace('\n','')
  text_input_ids = tokenizer.batch_encode_plus([text], return_tensors='pt')['input_ids'].to('cpu')

  summary_ids = model.generate(text_input_ids, num_beams=int(num_beams),
  length_penalty=float(length_penalty), 
  max_length=int(max_length),
  min_length=int(min_length), 
  no_repeat_ngram_size=int(no_repeat_ngram_size),
  top_k=50)

  summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
  return summary_txt

In [48]:
text = """
Johannes Gutenberg (1398 – 1468) was a German goldsmith and publisher who introduced printing to Europe. His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses.

Gutenberg many contributions to printing are: the invention of a process for mass-producing movable type, the use of oil-based ink for printing books, adjustable molds, and the use of a wooden printing press. His truly epochal invention was the combination of these elements into a practical system that allowed the mass production of printed books and was economically viable for printers and readers alike.

In Renaissance Europe, the arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society. The relatively unrestricted circulation of information—including revolutionary ideas—transcended borders, and captured the masses in the Reformation. The sharp increase in literacy broke the monopoly of the literate elite on education and learning and bolstered the emerging middle class.
"""

Johannes Gutenberg (1398 – 1468) was a German goldsmith and publisher who introduced printing to Europe. His introduction of mechanical movable type printing to Europe started the Printing Revolution and is widely regarded as the most important event of the modern period. It played a key role in the scientific revolution and laid the basis for the modern knowledge-based economy and the spread of learning to the masses.

Gutenberg many contributions to printing are: the invention of a process for mass-producing movable type, the use of oil-based ink for printing books, adjustable molds, and the use of a wooden printing press. His truly epochal invention was the combination of these elements into a practical system that allowed the mass production of printed books and was economically viable for printers and readers alike.

In Renaissance Europe, the arrival of mechanical movable type printing introduced the era of mass communication which permanently altered the structure of society. The relatively unrestricted circulation of information—including revolutionary ideas—transcended borders, and captured the masses in the Reformation. The sharp increase in literacy broke the monopoly of the literate elite on education and learning and bolstered the emerging middle class.

In [56]:
summarize(text, 4, 1.0, 50, 20, 4)

' Gutenberg was one of the founding fathers of the printing revolution in the 14th Century. and his work was credited with the invention of the printing system.'